In [66]:
import numpy as np
from qtft_tools import *
from scipy import sparse
import networkx as nx
from tqdm import tqdm
import community
import matplotlib.pyplot as plt
import seaborn as sns

from numpy import tensordot, kron, ones
from scipy.optimize import minimize, LinearConstraint, Bounds

# Q-Learning values. (Discounted Payoff action is in first column).
Qs = sparse.load_npz('data/Qs.npz').toarray() 

# Association matrix
As = sparse.load_npz('data/As.npz').toarray()

# Provision constraints
Ls = sparse.load_npz('data/Ls.npz').toarray()

# Number of assets
N = len(Qs)

# Number of actions
M = len(Qs[0,:])

# Number of candidate solutions from QAOA
n_candidates = 10

# Number of driving cycles in QAOA. The larger this value, the better and the slower the solutions 
p = 2

# Max community size in the divide-and-conquer algorithm. 
max_community_size = 7

# Weight (epsilon) in the objective function
e = 0.2


num = 50
Qs = Qs[:num,:]
As = As[:num,:num]
Ls = Ls[:num,:]
N = len(Qs)

print('Qs (Discounted Payoff) shape:', Qs.shape)
print('As (Association matrix) shape:', As.shape)
print('Ls (Provision constraints) shape:', Ls.shape)
print('N (Number of assets):', N)
print('M (Number of actions):', M)

print()
print("As:")
print(As)

Qs (Discounted Payoff) shape: (50, 5)
As (Association matrix) shape: (50, 50)
Ls (Provision constraints) shape: (50, 5)
N (Number of assets): 50
M (Number of actions): 5

As:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [72]:
dc_m = DC_QAOA(Qs, As, e, p, n_candidates, max_community_size)
#dc_m.optimized()
dc_m.set_communities()

tmp = 0
for g in dc_m.groups:
    g.show()
    tmp += g.size

tmp

id:0_0, nodes:[17 28 23], size:3
id:1_0, nodes:[ 1 22 31], size:3
id:5_0, nodes:[35 36], size:2
id:7_0, nodes:[34 47], size:2
id:6_0, nodes:[40 27], size:2
id:8_0, nodes:[49  6], size:2
id:4_0, nodes:[33  4], size:2
id:3_0, nodes:[26 13], size:2
id:2_0, nodes:[ 0 16], size:2
id:18_0, nodes:[14], size:1
id:9_0, nodes:[2], size:1
id:10_0, nodes:[3], size:1
id:11_0, nodes:[5], size:1
id:12_0, nodes:[7], size:1
id:13_0, nodes:[8], size:1
id:14_0, nodes:[9], size:1
id:15_0, nodes:[10], size:1
id:16_0, nodes:[11], size:1
id:17_0, nodes:[12], size:1
id:38_0, nodes:[48], size:1
id:37_0, nodes:[46], size:1
id:29_0, nodes:[37], size:1
id:36_0, nodes:[45], size:1
id:35_0, nodes:[44], size:1
id:34_0, nodes:[43], size:1
id:33_0, nodes:[42], size:1
id:32_0, nodes:[41], size:1
id:31_0, nodes:[39], size:1
id:30_0, nodes:[38], size:1
id:28_0, nodes:[32], size:1
id:20_0, nodes:[18], size:1
id:27_0, nodes:[30], size:1
id:26_0, nodes:[29], size:1
id:25_0, nodes:[25], size:1
id:24_0, nodes:[24], size:1
id:

50

In [125]:
dc_m = DC_QAOA(Qs, As, e, p, n_candidates, max_community_size)
#dc_m.optimized()
dc_m.set_communities()

#for i in tqdm(range(len(self.groups))):
i = 0
    
# Perform QAOA
Q = dc_m.Qs[dc_m.groups[i].nodes]
A = dc_m.As[dc_m.groups[i].nodes][:, dc_m.groups[i].nodes]    
m = QAOA(Q, A, dc_m.e, dc_m.p)
      
# to edit -------------------------------------------------------
m.optimized()

# Output probabilities
ps = np.abs(np.reshape(m.psi,-1))**2

# Save Candidates
inds = np.argsort(ps)[::-1][:dc_m.n_candidates]
dc_m.groups[i].ps = ps[inds]

# Save selected bitstrings
for ind in inds:
    dc_m.groups[i].xs = np.append(dc_m.groups[i].xs, m.get_str_from_index(ind))

# Save cost functions for selected bitstrings
for bitstring in dc_m.groups[i].xs:
    dc_m.groups[i].cs = np.append(dc_m.groups[i].cs, m.get_cost_from_str(bitstring))

# State reconstruction
if i == 0:
    dc_m.res = dc_m.groups[0]
else:
    dc_m.res = dc_m._combine_groups(dc_m.res, dc_m.groups[i], ps, m)



i = 1
    
# Perform QAOA
Q = dc_m.Qs[dc_m.groups[i].nodes]
A = dc_m.As[dc_m.groups[i].nodes][:, dc_m.groups[i].nodes]    
m = QAOA(Q, A, dc_m.e, dc_m.p)
      
# to edit -------------------------------------------------------
m.optimized()

# Output probabilities
ps = np.abs(np.reshape(m.psi,-1))**2

# Save Candidates
inds = np.argsort(ps)[::-1][:dc_m.n_candidates]
dc_m.groups[i].ps = ps[inds]

# Save selected bitstrings
for ind in inds:
    dc_m.groups[i].xs = np.append(dc_m.groups[i].xs, m.get_str_from_index(ind))

# Save cost functions for selected bitstrings
for bitstring in dc_m.groups[i].xs:
    dc_m.groups[i].cs = np.append(dc_m.groups[i].cs, m.get_cost_from_str(bitstring))

In [145]:
L = dc_m.res
R = dc_m.groups[i]
_, ind_L, ind_R = np.intersect1d(L.nodes, R.nodes, return_indices = True)
LR = Group(name = L.id+"|"+R.id, nodes = np.concatenate((L.nodes, np.delete(R.nodes, ind_R))))
# Loop over all candidates
for ii in range(len(L.xs)):
    for jj in range(len(R.xs)):
        LR = dc_m._combine_bitstrings(L.xs[ii], R.xs[jj], ind_L, ind_R, LR)

m2 = QAOA(dc_m.Qs[LR.nodes], dc_m.As[LR.nodes][:,LR.nodes], dc_m.e, dc_m.p)
for x in LR.xs:
    LR.cs = np.append(LR.cs, m2.get_cost_from_str(x))

In [135]:
LR.show()

id:0_0|1_0, nodes:[17 28 23  1 22 31], size:6


In [141]:
print(L.xs)
print(R.xs)
print(LR.xs)

['333' '330' '303' '033' '331' '233' '313' '300' '133' '030']
['000' '200' '002' '202' '004' '020' '400' '204' '402' '220']
['333000' '333200' '333002' '333202' '333004' '333020' '333400' '333204'
 '333402' '333220' '330000' '330200' '330002' '330202' '330004' '330020'
 '330400' '330204' '330402' '330220' '303000' '303200' '303002' '303202'
 '303004' '303020' '303400' '303204' '303402' '303220' '033000' '033200'
 '033002' '033202' '033004' '033020' '033400' '033204' '033402' '033220'
 '331000' '331200' '331002' '331202' '331004' '331020' '331400' '331204'
 '331402' '331220' '233000' '233200' '233002' '233202' '233004' '233020'
 '233400' '233204' '233402' '233220' '313000' '313200' '313002' '313202'
 '313004' '313020' '313400' '313204' '313402' '313220' '300000' '300200'
 '300002' '300202' '300004' '300020' '300400' '300204' '300402' '300220'
 '133000' '133200' '133002' '133202' '133004' '133020' '133400' '133204'
 '133402' '133220' '030000' '030200' '030002' '030202' '030004' '030020'


In [146]:
LR.cs

array([-1.68055473, -1.94730443, -1.94730443, -2.24482228, -1.603517  ,
       -1.59263179, -1.603517  , -1.90103485, -1.90103485, -1.85938149,
       -1.45644164, -1.72319134, -1.72319134, -2.02070919, -1.37940391,
       -1.3685187 , -1.37940391, -1.67692176, -1.67692176, -1.6352684 ,
       -1.28434029, -1.55109   , -1.55109   , -1.84860785, -1.20730257,
       -1.19641735, -1.20730257, -1.50482042, -1.50482042, -1.46316706,
       -1.29173635, -1.55848606, -1.55848606, -1.85600391, -1.21469863,
       -1.20381341, -1.21469863, -1.51221648, -1.51221648, -1.47056312,
       -1.41717835, -1.68392806, -1.68392806, -1.9814459 , -1.34014063,
       -1.32925541, -1.34014063, -1.63765848, -1.63765848, -1.59600512,
       -1.27882236, -1.54557207, -1.54557207, -1.84308992, -1.20178464,
       -1.19089942, -1.20178464, -1.49930249, -1.49930249, -1.45764913,
       -1.20743935, -1.47418906, -1.47418906, -1.77170691, -1.13040163,
       -1.11951641, -1.13040163, -1.42791948, -1.42791948, -1.38